# Chapter 25: Modules

---

## 25.1 Understanding Modules

TypeScript's module system is built on top of JavaScript's module patterns, providing a way to organize code into reusable, encapsulated units. Understanding how modules work—from detection to resolution—is fundamental for building scalable TypeScript applications.

### 25.1.1 Modules vs Scripts

TypeScript distinguishes between **modules** (files with import/export statements) and **scripts** (global files without module syntax). This distinction affects scope, variable visibility, and how the compiler processes the file.

```typescript
// Module file (has exports or imports)
// user.ts
export interface User {
  id: number;
  name: string;
}

export function createUser(name: string): User {
  return { id: Math.random(), name };
}

// Variables are module-scoped
const internalCache = new Map(); // Not visible outside this module
```

```typescript
// Script file (no exports/imports)
// global.d.ts or legacy file

// Variables are in global scope
declare var APP_VERSION: string;

// Without 'export', this augments global namespace
interface Window {
  customProperty: string;
}
```

**Key Differences:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Modules vs Scripts Comparison                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Feature           │ Module                      │ Script          │
│  ───────────────────┼─────────────────────────────┼──────────────── │
│   Scope             │ Module-scoped               │ Global          │
│   Variables         │ Must export to share        │ Implicitly      │
│                     │                             │ global          │
│   Strict mode       │ Always strict               │ Optional        │
│   Top-level await   │ Supported                   │ Not supported   │
│   File extension    │ .ts, .tsx, .js, .mjs        │ .ts, .js        │
│   Detection         │ Has import/export           │ No import/      │
│                     │                             │ export          │
│   tslib helpers     │ Imported per module         │ May be global   │
│                                                                     │
│   Module Detection:                                                  │
│   • File is a module if it has at least one top-level import       │
│     or export statement                                              │
│   • 'export {}' can force module mode without exporting anything     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 25.1.2 Module Resolution

Module resolution is the process of finding the actual file path for an import specifier. TypeScript supports multiple resolution strategies.

```typescript
// Relative imports (resolved relative to current file)
import { helper } from "./utils/helper";
import { config } from "../config/settings";

// Non-relative imports (resolved via node_modules or path mapping)
import React from "react";
import { merge } from "lodash";
import { internalUtil } from "@company/utils";

// URL imports (modern ES modules)
import { wasm } from "https://example.com/module.js";
```

**Resolution Process:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Module Resolution Flow                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Import: import { x } from "module-name"                            │
│                                                                     │
│   Step 1: Is it relative? (starts with ./ or ../)                   │
│   ┌───────────────────────────────────────────────────────────────┐ │
│   │ YES: Resolve relative to importing file                        │ │
│   │   • Try extensions: .tsx, .ts, .d.ts, .js, .json            │ │
│   │   • Try index files in directories                            │ │
│   └───────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Step 2: Check path mappings (tsconfig.json paths)                │
│   ┌───────────────────────────────────────────────────────────────┐ │
│   │ If match found, resolve to mapped path                        │ │
│   └───────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Step 3: Resolve as node_modules package                           │
│   ┌───────────────────────────────────────────────────────────────┐ │
│   │ • Look in node_modules/module-name                            │ │
│   │ • Check package.json "types" or "typings" field             │ │
│   │ • Check package.json "exports" field (modern)               │ │
│   │ • Look for index.d.ts or index.ts                           │ │
│   └───────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Step 4: Check global declarations (if allowSyntheticImports)      │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 25.2 Exporting

Export declarations make module members available for import by other modules. TypeScript supports multiple export styles to accommodate different patterns.

### 25.2.1 Named Exports

Named exports allow you to export multiple values from a module, each identified by its name.

```typescript
// math.ts - Multiple named exports
export const PI = 3.14159;
export const E = 2.71828;

export function add(a: number, b: number): number {
  return a + b;
}

export function subtract(a: number, b: number): number {
  return a - b;
}

// Exporting types
export interface Point {
  x: number;
  y: number;
}

export type Vector = {
  dx: number;
  dy: number;
};

// Exporting classes
export class Calculator {
  private value = 0;
  
  add(n: number): this {
    this.value += n;
    return this;
  }
  
  getValue(): number {
    return this.value;
  }
}

// Exporting constants with type annotations
export const ORIGIN: Point = { x: 0, y: 0 };

// Re-exporting from another module (re-export)
export { helper } from "./utils";
export { config as configuration } from "./config";
```

**Export List Syntax:**

```typescript
// Alternative syntax: export list at end of file
const API_URL = "https://api.example.com";
const TIMEOUT = 5000;

function fetchData() {
  // implementation
}

function processData() {
  // implementation
}

// Export list
export { API_URL, TIMEOUT, fetchData, processData };

// Rename exports
export { fetchData as getData, processData as transformData };
```

### 25.2.2 Default Exports

Default exports provide a single "main" export from a module. A module can have only one default export.

```typescript
// user.ts - Default export pattern
export interface User {
  id: string;
  name: string;
}

export class UserManager {
  private users: User[] = [];
  
  add(user: User): void {
    this.users.push(user);
  }
  
  getById(id: string): User | undefined {
    return this.users.find(u => u.id === id);
  }
}

// Default export (usually the primary class/function)
export default UserManager;

// Alternative: Anonymous default export
export default class {
  private data = "";
  
  getData() {
    return this.data;
  }
}

// Function as default export
// calculator.ts
export default function calculate(a: number, b: number): number {
  return a + b;
}

// Object as default export
// config.ts
export default {
  apiUrl: "https://api.example.com",
  version: "1.0.0",
  features: {
    auth: true,
    caching: false
  }
} as const;
```

**Best Practices for Defaults:**

```typescript
// ✅ DO: Use default export for the primary class/function of a module
// File: Calculator.ts
export default class Calculator {
  // implementation
}

// ✅ DO: Export types/interfaces as named exports alongside default
export interface CalculatorOptions {
  precision: number;
}

export default class Calculator {
  constructor(options: CalculatorOptions) {}
}

// ❌ DON'T: Mix default and named exports excessively
// This can be confusing:
export default function main() {}
export function helper1() {}
export function helper2() {}
export const constant = 42;

// ✅ DO: Consider using only named exports for libraries
// More explicit and tree-shakeable:
export { main, helper1, helper2, constant };
```

### 25.2.3 Re-exporting

Re-exports allow you to create "barrel" files that consolidate exports from multiple modules, providing a clean public API for packages.

```typescript
// index.ts - Barrel file pattern
export { User, UserManager } from "./user";
export { Product, ProductService } from "./product";
export { Order, OrderStatus } from "./order";

// Re-export with renaming
export { 
  User as UserModel,
  UserManager as UserController 
} from "./user";

// Re-export everything (namespace re-export)
export * from "./utils";
export * from "./types";

// Selective re-export with filtering
export { helper1, helper2 } from "./helpers";
// Don't export helper3 (internal use only)

// Re-export default (requires separate syntax)
export { default as UserService } from "./UserService";
export { default } from "./MainComponent"; // Re-export as this module's default
```

**Barrel File Pattern:**

```typescript
// utils/index.ts
export * from "./arrayUtils";
export * from "./stringUtils";
export * from "./dateUtils";
export * from "./validation";

// Consumer imports everything from one location:
// import { formatDate, validateEmail } from "./utils";
```

---

## 25.3 Importing

Import declarations bring exported members from other modules into the current scope. TypeScript provides several import styles for different use cases.

### 25.3.1 Named Imports

Named imports bring specific exports from a module into scope.

```typescript
// Importing specific names
import { User, UserManager } from "./user";
import { PI, add, subtract } from "./math";

// Importing with renaming (useful for avoiding conflicts)
import { User as UserModel } from "./models/User";
import { User as UserEntity } from "./entities/User";

// Use both:
const model: UserModel = { id: "1", name: "John" };
const entity: UserEntity = new UserEntity();

// Importing types (TypeScript-specific, removed at compile time)
import type { Point, Vector } from "./geometry";
import { type Point, type Vector } from "./geometry"; // TypeScript 4.5+ inline type imports

// Mixed value and type imports (TypeScript 4.5+)
import { Calculator, type CalculatorOptions } from "./Calculator";

// Import everything as namespace
import * as MathUtils from "./math";
console.log(MathUtils.PI);
console.log(MathUtils.add(1, 2));

// Import for side effects only (no imports, executes module)
import "./setup"; // Runs global setup code
import "./styles.css"; // Loads CSS (with appropriate loader)
```

### 25.3.2 Default Imports

Default imports bring the default export from a module.

```typescript
// Import default export
import UserManager from "./user";
import Calculator from "./calculator";
import config from "./config";

// Default import with named imports
import React, { useState, useEffect } from "react";
import express, { Request, Response } from "express";

// Default import with renaming
import MyCalculator from "./calculator";
import AppConfig from "./config";

// Importing CommonJS default (interop)
import React = require("react"); // CommonJS style
import * as React from "react";   // ES module style (works with CJS too)
```

### 25.3.3 Dynamic Imports

Dynamic imports load modules asynchronously at runtime, returning a Promise. This enables code splitting and lazy loading.

```typescript
// Static imports (loaded at compile time)
import { heavyFunction } from "./heavy-module";

// Dynamic imports (loaded at runtime)
async function loadHeavyModule() {
  // Returns Promise<ModuleType>
  const module = await import("./heavy-module");
  module.heavyFunction();
  
  // Destructure from dynamic import
  const { heavyFunction, heavyClass } = await import("./heavy-module");
  heavyFunction();
  
  // Conditional loading
  if (userPrefersFeature) {
    const { FeatureModule } = await import("./feature-module");
    const feature = new FeatureModule();
    feature.initialize();
  }
  
  // Lazy loading with React
  const LazyComponent = React.lazy(() => import("./HeavyComponent"));
}

// Dynamic import with type annotations
async function loadUserModule() {
  // TypeScript infers the module shape
  const userModule = await import("./user");
  
  // Explicit typing (less common)
  const { UserManager }: typeof import("./user") = await import("./user");
}

// Import assertions (for JSON modules)
const jsonModule = await import("./data.json", {
  assert: { type: "json" }
});
```

**Dynamic Import Patterns:**

```typescript
// Route-based code splitting
const routes = {
  "/dashboard": () => import("./pages/Dashboard"),
  "/profile": () => import("./pages/Profile"),
  "/settings": () => import("./pages/Settings")
};

async function navigateTo(path: string) {
  const loader = routes[path as keyof typeof routes];
  if (loader) {
    const module = await loader();
    module.render();
  }
}

// Loading with timeout
async function importWithTimeout<T>(
  path: string,
  timeoutMs: number
): Promise<T> {
  return Promise.race([
    import(path),
    new Promise((_, reject) => 
      setTimeout(() => reject(new Error("Import timeout")), timeoutMs)
    )
  ]);
}
```

---

## 25.4 Module Resolution Strategies

TypeScript supports different module resolution strategies to accommodate various project structures and runtime environments.

### 25.4.1 Classic Resolution (Legacy)

The original TypeScript module resolution, used primarily for AMD and SystemJS modules.

```typescript
// Classic resolution (simplified)
// Import: import { x } from "module"
// Resolution:
// 1. Look for module.ts, module.tsx, module.d.ts
// 2. Look in node_modules/module (non-recursive)
// 3. Not commonly used in modern projects
```

**Configuration:**
```json
{
  "compilerOptions": {
    "moduleResolution": "classic" // Rarely used today
  }
}
```

### 25.4.2 Node.js Resolution

The standard resolution algorithm compatible with Node.js CommonJS and ES Modules.

```typescript
// Node.js resolution for relative paths
import { helper } from "./utils/helper";

// Resolution steps for "./utils/helper":
// 1. Check if file exists: utils/helper.ts, utils/helper.tsx
// 2. Check: utils/helper.d.ts
// 3. Check as directory: utils/helper/package.json "types" field
// 4. Check: utils/helper/index.ts, utils/helper/index.tsx, utils/helper/index.d.ts

// Node.js resolution for non-relative paths
import { merge } from "lodash";

// Resolution steps for "lodash":
// 1. Walk up directory tree looking for node_modules/lodash
// 2. Check node_modules/lodash/package.json "types" or "typings" field
// 3. Check node_modules/lodash/index.d.ts
// 4. Check node_modules/@types/lodash/index.d.ts
```

**Configuration:**
```json
{
  "compilerOptions": {
    "moduleResolution": "node", // Default for CommonJS
    "esModuleInterop": true,
    "allowSyntheticDefaultImports": true
  }
}
```

### 25.4.3 Node16/NodeNext Resolution

Modern resolution supporting ES Modules in Node.js 12+, respecting package.json "exports" and "imports" fields.

```typescript
// Supports package.json "exports" field
// Package structure:
// my-package/
//   package.json
//   src/
//     index.ts
//     utils.ts

// package.json with exports:
{
  "name": "my-package",
  "exports": {
    ".": {
      "import": "./dist/index.mjs",
      "require": "./dist/index.cjs",
      "types": "./dist/index.d.ts"
    },
    "./utils": {
      "import": "./dist/utils.mjs",
      "require": "./dist/utils.cjs",
      "types": "./dist/utils.d.ts"
    }
  }
}

// Consumer can import subpaths:
import { something } from "my-package";
import { util } from "my-package/utils";
```

**Configuration:**
```json
{
  "compilerOptions": {
    "module": "Node16", // or "NodeNext"
    "moduleResolution": "Node16", // Must match module setting
    "target": "ES2022"
  }
}
```

### 25.4.4 Bundler Resolution

Optimized for modern bundlers (Webpack, Vite, Rollup), supporting features like extensionless imports and custom alias resolution.

```typescript
// Bundler features
import { helper } from "./utils/helper"; // No extension needed
import { Component } from "@/components/Button"; // Path aliases
import { asset } from "~/assets/image.svg?url"; // Query parameters

// TypeScript recognizes these patterns with:
{
  "compilerOptions": {
    "moduleResolution": "bundler",
    "allowImportingTsExtensions": true,
    "resolveJsonModule": true,
    "paths": {
      "@/*": ["./src/*"],
      "~/*": ["./public/*"]
    }
  }
}
```

---

## 25.5 Path Mappings and Aliases

Path mappings allow you to configure custom aliases for module paths, simplifying imports and enabling more maintainable project structures.

### 25.5.1 Base URL Configuration

The `baseUrl` option establishes the root directory for non-relative module resolution.

```typescript
// tsconfig.json
{
  "compilerOptions": {
    "baseUrl": "./src",
    "paths": {
      // Aliases relative to baseUrl
      "@components/*": ["components/*"],
      "@utils/*": ["utils/*"],
      "@types/*": ["types/*"],
      "@/*": ["*"] // Root alias
    }
  }
}

// With baseUrl: "./src"
// File: src/features/user/profile.ts
import { Button } from "@components/Button"; // Resolves to src/components/Button
import { formatDate } from "@utils/date";    // Resolves to src/utils/date
import { User } from "@/types";              // Resolves to src/types
```

### 25.5.2 Path Mappings Configuration

The `paths` compiler option maps import patterns to concrete file paths.

```typescript
// tsconfig.json
{
  "compilerOptions": {
    "baseUrl": ".",
    "paths": {
      // Simple alias
      "@api/*": ["src/api/*"],
      
      // Multiple fallback locations
      "@shared/*": ["src/shared/*", "packages/shared/*"],
      
      // Exact module mapping
      "my-package": ["./local-packages/my-package/src"],
      
      // Scoped package mapping
      "@company/*": ["packages/*/src"],
      
      // Path mapping for types
      "custom-types": ["./types/index.d.ts"]
    }
  }
}

// Usage in code:
import { fetchUser } from "@api/users";
import { Button } from "@shared/components/Button";
import { internal } from "my-package";
import type { Config } from "custom-types";
```

**Practical Path Mapping Setup:**

```json
{
  "compilerOptions": {
    "target": "ES2022",
    "module": "ESNext",
    "moduleResolution": "bundler",
    "baseUrl": ".",
    "paths": {
      // Source aliases
      "@/*": ["src/*"],
      "@components/*": ["src/components/*"],
      "@hooks/*": ["src/hooks/*"],
      "@utils/*": ["src/utils/*"],
      "@services/*": ["src/services/*"],
      "@types/*": ["src/types/*"],
      
      // Test aliases
      "@test/*": ["test/*"],
      "@fixtures/*": ["test/fixtures/*"],
      
      // Feature-based aliases (for large apps)
      "@features/*": ["src/features/*"],
      "@user/*": ["src/features/user/*"],
      "@order/*": ["src/features/order/*"]
    }
  },
  "include": ["src/**/*", "test/**/*"]
}
```

### 25.5.3 Path Aliases in Practice

Using path aliases effectively requires configuration in both TypeScript and your build tool.

```typescript
// Vite configuration (vite.config.ts)
import { defineConfig } from 'vite';
import path from 'path';

export default defineConfig({
  resolve: {
    alias: {
      '@': path.resolve(__dirname, './src'),
      '@components': path.resolve(__dirname, './src/components'),
      '@utils': path.resolve(__dirname, './src/utils')
    }
  }
});

// Webpack configuration
module.exports = {
  resolve: {
    alias: {
      '@': path.resolve(__dirname, 'src'),
      '@components': path.resolve(__dirname, 'src/components')
    }
  }
};

// Usage stays clean regardless of file location:
// src/features/user/profile.tsx
import { Avatar } from "@components/Avatar";
import { formatDate } from "@utils/date";
import { useAuth } from "@/hooks/useAuth"; // Relative to src root
```

---

## 25.6 ES Modules vs CommonJS

Understanding the differences between ES Modules (ESM) and CommonJS (CJS) is crucial for interoperability and modernizing codebases.

### 25.6.1 ES Module Syntax

ES Modules are the standard in modern JavaScript, supporting static analysis and tree-shaking.

```typescript
// ES Module exports
export const foo = "foo";
export function bar() { return "bar"; }
export default class Main { }

// ES Module imports
import { foo, bar } from "./module";
import Main from "./module";
import * as namespace from "./module";
import { type SomeType } from "./module";

// Dynamic import (always returns Promise)
const module = await import("./module");
```

**ESM Characteristics:**
- Static imports (top-level only)
- Immutable bindings (imports cannot be reassigned)
- Top-level await supported
- Strict mode by default
- Tree-shakeable (dead code elimination)

### 25.6.2 CommonJS Syntax

CommonJS is Node.js's original module system, using runtime `require()` and `module.exports`.

```typescript
// CommonJS exports
exports.foo = "foo";
exports.bar = function() { return "bar"; };
module.exports = class Main { };

// CommonJS imports
const { foo, bar } = require("./module");
const Main = require("./module");
const namespace = require("./module");

// Dynamic require (synchronous)
const module = require("./module");
```

**CJS Characteristics:**
- Runtime require (can be conditional)
- Mutable exports (can reassign module.exports)
- Synchronous loading
- Non-strict by default (unless "use strict")
- Harder to tree-shake

### 25.6.3 Interoperability

TypeScript provides options for smooth interoperability between ESM and CJS.

```typescript
// tsconfig.json for ESM output with CJS interop
{
  "compilerOptions": {
    "module": "ESNext",
    "moduleResolution": "node",
    "esModuleInterop": true,
    "allowSyntheticDefaultImports": true,
    "target": "ES2022"
  }
}
```

**Importing CJS from ESM:**

```typescript
// TypeScript handles the interop
import React from "react"; // CJS module, works with esModuleInterop
import * as React from "react"; // Namespace import
import { useState } from "react"; // Named import (with allowSyntheticDefaultImports)

// For modules without default export:
import * as lodash from "lodash";
// or
import lodash = require("lodash"); // Traditional TypeScript import
```

**Exporting for Both:**

```typescript
// Dual module support (package.json)
{
  "name": "my-package",
  "main": "./dist/index.cjs",
  "module": "./dist/index.mjs",
  "types": "./dist/index.d.ts",
  "exports": {
    ".": {
      "import": "./dist/index.mjs",
      "require": "./dist/index.cjs",
      "types": "./dist/index.d.ts"
    }
  }
}

// TypeScript source (works for both):
export function greet(name: string): string {
  return `Hello, ${name}!`;
}

export default greet;
```

**Conditional Exports Pattern:**

```typescript
// package.json for dual-mode package
{
  "name": "my-lib",
  "version": "1.0.0",
  "type": "module", // Default to ESM
  "exports": {
    ".": {
      "import": {
        "types": "./dist/index.d.ts",
        "default": "./dist/index.js"
      },
      "require": {
        "types": "./dist/index.d.cts",
        "default": "./dist/index.cjs"
      }
    }
  },
  "scripts": {
    "build": "tsc && tsc -p tsconfig.cjs.json"
  }
}
```

---

## 25.7 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we covered TypeScript's module system comprehensively:

**Key Takeaways:**

1. **Module Fundamentals**:
   - Modules are files with import/export statements
   - Module scope vs global script scope
   - Module detection and the `export {}` pattern

2. **Export Patterns**:
   - Named exports for multiple values
   - Default exports for single primary exports
   - Re-exports (barrel files) for organizing public APIs
   - Type-only exports for cleaner output

3. **Import Patterns**:
   - Named imports with destructuring
   - Default imports for primary exports
   - Namespace imports (`import * as`)
   - Dynamic imports for code splitting
   - Import assertions for JSON/modules

4. **Module Resolution**:
   - Classic (legacy), Node.js, Node16/NodeNext, and Bundler strategies
   - Package.json exports field support
   - Resolution algorithm and file extension handling

5. **Path Mapping**:
   - `baseUrl` for root-relative imports
   - `paths` for custom aliases
   - Configuration in build tools (Vite, Webpack)
   - Best practices for large project organization

6. **ESM vs CJS**:
   - Syntax differences and characteristics
   - Interoperability with `esModuleInterop`
   - Dual-mode package configuration
   - Migration strategies

### Practical Exercises

**Exercise 1: Export Patterns**

Create a well-structured module:

```typescript
// Create a math utilities module (math.ts) that:
// 1. Exports named constants: PI, E
// 2. Exports named functions: add, subtract, multiply, divide
// 3. Exports a Calculator class as default
// 4. Exports types: Operation, CalculatorOptions
// 5. Creates an index.ts barrel file that re-exports everything

// Then create a consumer file that:
// 1. Imports the default Calculator
// 2. Imports specific named functions
// 3. Imports everything as a namespace
// 4. Uses dynamic import to load the module conditionally
```

**Exercise 2: Module Resolution**

Configure a project with path mappings:

```typescript
// Set up tsconfig.json with:
// 1. baseUrl pointing to project root
// 2. Path aliases:
//    - "@/*" -> "src/*"
//    - "@components/*" -> "src/components/*"
//    - "@utils/*" -> "src/utils/*"
//    - "@types/*" -> "src/types/*"

// Create files:
// src/components/Button.ts
// src/utils/format.ts
// src/types/index.ts

// Then import from a deeply nested file:
// src/features/user/settings/profile.ts
// Should be able to import:
// import { Button } from "@components/Button";
// import { formatDate } from "@utils/format";
// import { User } from "@types";
```

**Exercise 3: ESM/CJS Interoperability**

Create a dual-mode package:

```typescript
// 1. Create a package that exports both ESM and CJS:
//    - Source: TypeScript files
//    - Output: Both .mjs (ESM) and .cjs (CommonJS)
//    - Types: .d.ts for both

// 2. Configure package.json with:
//    - exports field for conditional exports
//    - types field pointing to correct declarations
//    - type: "module" or appropriate configuration

// 3. Write code that:
//    - Exports named functions
//    - Exports a default class
//    - Can be imported by both ESM and CJS consumers

// 4. Test imports:
//    - ESM: import { fn } from "package";
//    - CJS: const { fn } = require("package");
```

**Exercise 4: Dynamic Imports**

Implement code splitting:

```typescript
// Create a feature-based lazy loading system:

// 1. Define feature modules:
//    - features/admin.ts (heavy admin functionality)
//    - features/analytics.ts (charting library)
//    - features/export.ts (PDF generation)

// 2. Create a feature loader that:
//    - Accepts feature name as parameter
//    - Dynamically imports the correct module
//    - Returns typed module exports
//    - Handles loading errors gracefully
//    - Implements loading timeout

// 3. Type the dynamic import properly:
//    - Use ReturnType or typeof import()
//    - Ensure type safety of loaded modules
//    - Handle potential undefined cases
```

**Exercise 5: Declaration Merging with Modules**

Combine modules with declaration merging:

```typescript
// 1. Create a module that exports a class 'PluginManager'
//    - Also merge with namespace to add static methods
//    - Static methods: register(), getAll(), clear()

// 2. Create module augmentation:
//    - Augment an existing library type (e.g., Express Request)
//    - Add custom properties that your middleware provides
//    - Ensure type safety when accessing augmented properties

// 3. Create a barrel file that:
//    - Re-exports from multiple sub-modules
//    - Organizes exports by category (types, utils, components)
//    - Uses re-export renaming to create clean API
```

### Additional Resources

- **TypeScript Handbook - Modules**: https://www.typescriptlang.org/docs/handbook/modules.html
- **TypeScript Module Resolution**: https://www.typescriptlang.org/docs/handbook/module-resolution.html
- **Node.js ES Modules**: https://nodejs.org/api/esm.html
- **Package.json Exports Field**: https://nodejs.org/api/packages.html#exports
- **TypeScript Path Mapping**: https://www.typescriptlang.org/docs/handbook/module-resolution.html#path-mapping

---

## Coming Up Next: Chapter 26 - Namespaces

In the next chapter, we will explore **Namespaces** (formerly internal modules), TypeScript's way of organizing code within a single file or across related files:

- **26.1 Understanding Namespaces** - Organizing code with internal modules
- **26.2 Defining Namespaces** - Syntax and structure
- **26.3 Namespace Imports** - Referencing external namespaces
- **26.4 Aliases in Namespaces** - Working with long namespace chains
- **26.5 When to Use Namespaces** - Comparing with ES Modules

While ES Modules have largely replaced namespaces for external module organization, namespaces remain valuable for specific use cases like organizing code within large files, declaring non-JavaScript values, and certain design patterns. Understanding when and how to use namespaces completes your TypeScript module system knowledge.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../6. advanced_type_features/24. declaration_merging.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='26. namespaces.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
